In [3]:
import os
import html
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
plt.style.use("ggplot")

import os.path as ospath

import nltk.data
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from operator import itemgetter

from gensim.models import word2vec
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans

import string
from time import time

from bs4 import BeautifulSoup  
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag

# import initial modules
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

C:\Users\10252946\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\10252946\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Read in the dataset

In [5]:
# path of the current directory
cur_dir = os.getcwd()
print("Current directory: {}".format(cur_dir))
# list files in the current directory
#print("Files in the current directory: {}".format(os.listdir(cur_dir)))
data_path = cur_dir+"\drugsCom"
train_path = data_path + "\drugsComTrain_raw.tsv"
test_path = data_path + "\drugsComTest_raw.tsv"
#rawdata_path = cur_dir+"/drugsCom"
#metadata_path = cur_dir+"/meta_Health_and_Personal_Care.json.gz"
print("Path of the raw data: {}".format(data_path))
print("Files in the drugCom directory: {}".format(os.listdir(data_path)))

Current directory: C:\Users\10252946\Documents\Education\GATech\BigDataForHealth\Projects
Path of the raw data: C:\Users\10252946\Documents\Education\GATech\BigDataForHealth\Projects\drugsCom
Files in the drugCom directory: ['drugsComTest_raw.tsv', 'drugsComTrain_raw.tsv']


In [6]:
train = pd.read_table(train_path, sep='\t')
test = pd.read_table(test_path, sep='\t')

In [7]:
train.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [14]:
train.shape

(161297, 7)

In [15]:
train['drugName'].nunique()

3436

## 1. Data preprocessing

#### 1.1 Define feature variables

In [16]:
# Take the values of the column that contains review text data, save to a variable named "documents"
documents = train['review'].values

In [9]:

# define a function to clean review
def review_to_words(raw_review):
    # input : raw_string (e.g. train['review'][0])
    # output : string consisting of words (preprocessed review)
    
    # remove HTML tags with BeautifulSoup
    review_text = BeautifulSoup(raw_review).get_text()
    
    # remove numbers and punctuation 
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    
    # split string into list of words 
    letters_only = letters_only.lower().split()
    
    # remove stop words 
    stops = set(stopwords.words('english'))
    meaningful_words = [word for word in letters_only if word not in stops]
    
    # join the words into single string and return it as a result
    cleaned_review = ' '.join(meaningful_words)
    
    return cleaned_review

In [10]:
# Clean the review field
review_to_words(train['review'][0])

'side effect take combination bystolic mg fish oil'

In [12]:
# apply the function to all raw reviews
reviews_number = train.shape[0]
print('cleaning up raw reviews...')
train['cleaned_reviews'] = train['review'].apply(review_to_words)
print('finished')

cleaning up raw reviews...


finished


In [17]:
documents
train['cleaned_reviews'].head()



KeyError: 'cleaned_reviews'

In [8]:
from sklearn.cross_validation import train_test_split

In [9]:
# X: documents
# Y: targets
# Now split the data to training set 80% and test set 20%
documents_train, documents_test = train_test_split(documents, test_size = 0.2, random_state = 42)

NameError: name 'documents' is not defined

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
# Create TfidfVectorizer, and name it vectorizer
# choose a reasonable max_features, e.g. 1000 to fast the computation speed
vectorizer = TfidfVectorizer(stop_words = 'english', max_features = 1000)

In [14]:
# Train the model with your training data
vectors_train = vectorizer.fit_transform(documents_train).toarray()

In [15]:
# Get the vocab of your tfidf
words = vectorizer.get_feature_names()

In [16]:
# Use the trained model to transform all the reviews
vectors_documents = vectorizer.transform(documents).toarray()


Second technique for designing features using Count Vectorizer


In [27]:
from sklearn.feature_extraction.text import CountVectorizer


In [29]:
countvectorizer = CountVectorizer(analyzer='word', tokenizer=None, 
                             preprocessor=None, stop_words=None, 
                             max_features=5000)

cleaned_reviews = train['cleaned_reviews'].tolist()
train_data_features = countvectorizer.fit_transform(cleaned_reviews)
train_data_features = train_data_features.toarray()

# enumerate counts of each word in the all reviews in descending order

In [30]:
vocab = countvectorizer.get_feature_names()
counts = np.sum(train_data_features, axis=0)

word_count = list()
for word, count in zip(vocab, counts):
        word_count.append([word, count])
wordcount = sorted(word_count, key=itemgetter(1))[::-1]

In [31]:
wordcount[:100]

[['day', 57908],
 ['side', 53583],
 ['taking', 51307],
 ['years', 46891],
 ['pain', 45704],
 ['mg', 45644],
 ['effects', 45227],
 ['first', 45052],
 ['take', 43984],
 ['months', 43010],
 ['started', 42351],
 ['like', 41951],
 ['get', 41398],
 ['days', 40569],
 ['time', 39021],
 ['feel', 35520],
 ['would', 33571],
 ['back', 31475],
 ['pill', 31163],
 ['one', 30428],
 ['weeks', 29797],
 ['week', 28677],
 ['also', 28303],
 ['doctor', 28134],
 ['took', 28108],
 ['period', 27998],
 ['medication', 27646],
 ['weight', 27524],
 ['got', 27235],
 ['month', 26710],
 ['life', 25506],
 ['since', 25451],
 ['bad', 24607],
 ['still', 24529],
 ['anxiety', 24140],
 ['really', 23789],
 ['much', 23470],
 ['never', 22462],
 ['better', 22058],
 ['went', 21999],
 ['medicine', 21631],
 ['two', 21559],
 ['work', 21463],
 ['felt', 20697],
 ['well', 20494],
 ['control', 20463],
 ['every', 20330],
 ['good', 19985],
 ['year', 19568],
 ['great', 19283],
 ['even', 19282],
 ['night', 19056],
 ['go', 18719],
 ['sleep'

In [17]:
from sklearn.cluster import KMeans

kmeans = KMeans()

kmeans.fit(vectors_train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=8, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [27]:
assigned_cluster = kmeans.predict(vectors_documents)

In [28]:
# Default of kmeans uses 8 clusters
print ('number of clusters:' + str(kmeans.cluster_centers_.shape))

number of clusters:(8, 1000)


In [29]:
# print top 10 words of each cluster centers
# step (1) Sort each centroid vector to find the top 10 features
top_centroids = kmeans.cluster_centers_.argsort()[:, -1:-11:-1]
print("top 10 features for each cluster:")
# step (2) Go back to our vectorizer object to find out what words each of these features corresponds to
for num, centroid in enumerate(top_centroids):
    print("%d: %s" % (num, ", ".join(words[i] for i in centroid)))

top 10 features for each cluster:
0: pain, 039, day, relief, severe, years, taking, medicine, medication, doctor
1: day, 039, sleep, night, took, taking, days, hours, like, felt
2: period, 039, pill, birth, control, bleeding, periods, months, month, weight
3: quot, 039, day, years, like, feel, effects, taking, just, time
4: anxiety, 039, panic, depression, attacks, life, feel, taking, medication, effects
5: effects, years, 039, works, medicine, medication, taking, great, drug, life
6: 039, ve, don, like, feel, just, taking, effects, years, weight
7: acne, skin, face, 039, clear, using, months, dry, month, cleared


In [30]:
# Find the top 10 features for each cluster.
kmeans = KMeans(n_clusters = 5)
kmeans.fit(vectors_train)
assigned_cluster = kmeans.predict(vectors_documents)

top_centroids = kmeans.cluster_centers_.argsort()[:, -1:-11:-1]
print("top 10 features for each cluster:")
for num, centroid in enumerate(top_centroids):
    print("%d: %s" % (num, ",".join(words[i] for i in centroid)))

top 10 features for each cluster:
0: anxiety,039,depression,panic,attacks,life,feel,taking,medication,day
1: 039,ve,don,like,day,feel,just,taking,effects,years
2: pain,039,day,relief,severe,taking,years,medicine,medication,doctor
3: 039,day,effects,taking,years,medication,medicine,works,started,days
4: 039,period,pill,birth,control,periods,months,bleeding,month,weight


<b>Print out the rating and review of a random sample of the reviews assigned to each cluster to get a sense of the cluster.</b>

In [35]:
for i in range(kmeans.n_clusters):
    cluster = np.arange(0, vectors_documents.shape[0])[assigned_cluster==i]
    sample_reviews = np.random.choice(cluster, 1, replace=False)
    print("cluster %d:" % i)
    for review in sample_reviews:
        print("    %s" % train.loc[review]['review'])

cluster 0:
    "I suffered with a pretty severe anxiety, stress and depression. Took Paxil 4 days ago and honestly I feel great. It already started working. Lot of it could be in my head but trust me all my symptoms disappeared. Side effects: from day one I had a light headache, day two -a little harder headache, day three unbearable headache, day four(now) no headache, feeling perfectly fine, great mood, even though its raining and dark outside.  GREAT medicine. Also its worth to add, it worked the same wonders for me about 8 years ago when I had it for the last time. I was on Paxil for 6 months and it healed me. I had 7 years of great life and suddenly 6-7 months ago this thing attacked me again. Paxil works again, same way."
cluster 1:
    "[History]- I had started taking melatonin (3mg) to help me sleep, shortly after my divorce. As my body &#039;adjusted&#039; to it, I began to increase the dose, eventually (over several years) working up to 30mg. Well, not too long after that, I 

In [38]:
# Find the drugname which got most reviews, get your filtered df, name it df_top_drug
df_top_drug = train['drugName'].value_counts().index[0]
df_top_drug


'Levonorgestrel'

In [49]:
train.shape

(161297, 7)

In [47]:
train['condition'].nunique()

884

In [50]:
train['condition'][:40]

0     Left Ventricular Dysfunction
1                             ADHD
2                    Birth Control
3                    Birth Control
4                Opiate Dependence
5     Benign Prostatic Hyperplasia
6          Emergency Contraception
7                  Bipolar Disorde
8                         Epilepsy
9                    Birth Control
10             Migraine Prevention
11                      Depression
12                 Crohn's Disease
13                           Cough
14                   Birth Control
15                         Obesity
16         Urinary Tract Infection
17                     ibromyalgia
18                 Bipolar Disorde
19    Chronic Myelogenous Leukemia
20                   HIV Infection
21                        Insomnia
22                   Birth Control
23            Rheumatoid Arthritis
24         Vaginal Yeast Infection
25             Chlamydia Infection
26                       Hirsutism
27                            ADHD
28                  

In [ ]:
# dendrogram

In [45]:
train.groupby('drugName').size()

drugName
A + D Cracked Skin Relief                              1
A / B Otic                                             1
Abacavir / dolutegravir / lamivudine                  52
Abacavir / lamivudine / zidovudine                     1
Abatacept                                             19
Abilify                                              446
Abilify Discmelt                                       2
Abilify Maintena                                       4
Abiraterone                                            9
AbobotulinumtoxinA                                     3
Abraxane                                               4
Abreva                                               158
Absorbine Jr.                                          1
Absorica                                               2
Acamprosate                                          109
Acanya                                                45
Acarbose                                               1
Accolate              

    1) Use all the reviews provided along with the Word2vec library in order to train the word vectors.
    2) Try different approaches to feature creation such as :
        2.1) Averaging/ Mean of all word vectors for each review.
        2.2) Root mean square of all word vectors for each review.
        2.3) Standard deviation of all word vectors for each review
        2.4) Clustering all the words and using cluster count in each reciew as feature.
    3) Once the features are available, we use a classification technique, in this case a random forest classifier to 
    classify sentiment of the reviews as Good or Bad.

Detailed explanation :-
        1) Training the word2vec model:
            The data source at "https://www.kaggle.com/c/word2vec-nlp-tutorial/data" provides three basic data sets of
            movie reviews - labeledTrainData.tsv which is a lablelled set of 25000 reviews as well as their sentiment,
            testData.tsv which is an unlabled set of 25000 reviews and unlabeledTrainData.tsv which is also an unlabelled
            set of 50000 reviews. I have used all of these sets to train the word2vec model since more the data, the better
            the word vectors. This is done after cleaning the reviews such as removing punctuations, URLs etc.
        2) Exploring different approaches to Feature creation:
            To study the accuracy, I have chosen to use only the labeledTrainData.tsv file which is a lablelled set of 
            25000 reviews as well as their sentiment. I have split this set into 22000 records of training data and the
            remaining 3000 records will be used as my test data.
            The approaches I have tried for feature creation are as follows : 
                2.1) Averaging/ Mean of all word vectors for each review.
                    Each review has many words. We average out the word vectors for each word to create the feature vector
                    for each review.
                2.2) Root mean square of all word vectors for each review.
                    Each review has many words. We compute the Root mean square of word vectors for each word to create 
                    the feature vector for each review.
                2.3) Standard deviation of all word vectors for each review
                    Each review has many words. We compute the Standard deviation of word vectors for each word to create 
                    the feature vector for each review.
                2.4) Clustering all the words and using cluster count in each reciew as feature.
                    Here, we use K-means clustering to cluster all the words in the vocabulary of the trained word2vec
                    model. Features for each review are a vector of how many words from each of these clusters belong to
                    the review.
            3) Once the features are available, we use a classification technique, in this case a random forest classifier to 
                classify sentiment of the reviews as Good or Bad. We train the classifier using the 22000 labelled records
                and test accuracy of each feature creation method on the 3000 test data.

1) Use all the reviews provided along with the Word2vec library in order to train the word vectors.
    2) Try different approaches to feature creation such as :
        2.1) Averaging/ Mean of all word vectors for each review.
        2.2) Root mean square of all word vectors for each review.
        2.3) Standard deviation of all word vectors for each review
        2.4) Clustering all the words and using cluster count in each reciew as feature.
    3) Once the features are available, we use a classification technique, in this case a random forest classifier to 
    classify sentiment of the reviews as Good or Bad.

In [10]:
def cleanReviewSentences(sentence, removeStopwords=False, stopWords=None):
    """
        A function that takes a review string and cleans it by removing HTML formatting, any non alpha-numeric character
        , convert to lower case and if requested remove stop words respectively. This function returns a list of clean words.
    """

    cleanedSentence = (BeautifulSoup(sentence)).get_text()

    cleanedSentence = re.sub("[^a-zA-Z0-9 ]", " ", cleanedSentence)

    cleanedWords = (cleanedSentence.lower()).split()

    cleanedWords = [word for word in cleanedWords if len(word) > 0]

    if removeStopwords == True:
        cleanedWords = [word for word in cleanedWords if not word in stopWords]

    return cleanedWords

In [11]:
def preprossesReviews(review, tokenizer):
    """
        This function cleans the reviews by removing any URLs from it, then splitting the review into sentence and then
        cleaning each sentence by calling "cleanReviewSentences". This function returns a list of all cleaned words in the
        review.
    """

    cleanedListOfSentences = []

    review = re.sub("http:[a-zA-Z0-9/\.]*", " ", review)

    reviewSentences = tokenizer.tokenize(review.strip())

    for sentence in reviewSentences:
        if len(sentence) > 0:
            cleanedListOfSentences.append(cleanReviewSentences(sentence))

    return cleanedListOfSentences

In [12]:
def printProgress(percent, message=""):
    """A sinmple function to print progress for time consuming operations"""
    sys.stdout.write(
        "\r\t" + message + "\t[" + "*" * round(percent / 10) + " " * (10 - round(percent / 10)) + "]  " + str(
            int(percent)) + "% Completed.")

In [13]:
def prepareTrainingData(reviews, tokenizer):
    """
        This function is used to clean all the reviews to be used for training the word2vec model. This function calls
        "preprossesReviews" for each review.
    """
    cleanedListOfSentences = []
    i = 0
    dataSize = len(reviews)
    for review in reviews:
        if i % 100 == 0:
            printProgress(((i + 1) * 100) / dataSize, "Preparing data for training word2vec ...")
        cleanedListOfSentences += preprossesReviews(review, tokenizer)
        i += 1
    printProgress(100)

    return cleanedListOfSentences

In [14]:
def wordVectorTraining(numberOfFeatures=300):
    """
        This function is used to train the word2vec model or load an already trained model if it exists.
    """
    modelName = "wordVectormodel"
    if ospath.isfile("wordVectormodel") == True:
        model = word2vec.Word2Vec.load(modelName)
    else:
        minWordCount = 40
        numberOfWorkers = 4
        context = 10
        downSampling = 1e-3
    
        tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        wordVecTrainingData = setUpDataForWordVectors(tokenizer)
    
        model = word2vec.Word2Vec(wordVecTrainingData, workers=numberOfWorkers, size=numberOfFeatures,
                                  min_count=minWordCount, window=context, sample=downSampling)
        model.init_sims(replace=True)
    
        model.save(modelName)

    return model
        

In [15]:
def createFeatureFromReviewWordVectors(reviewWords, vocabulary, model, numberOfFeatures):
    """
        This is a very important function that creates the feature vectors by using mean, root mean square or standard
        deviation of all the word vectors in each review.
    """

    featureVector = np.zeros((1, numberOfFeatures), dtype=np.float32)[0]

    numberOfWords = 0

    if type == 1 or type == 3:
        """This is feature creation using mean of all word vectors which gave a test data accuracy of 83.1%"""
        for word in reviewWords:
            if word in vocabulary:
                numberOfWords = numberOfWords + 1.
                featureVector = np.add(featureVector, model[word])

        featureVector = np.divide(featureVector, numberOfWords)

    if type == 2:
        """This is feature creation using root mean square of all word vectors which gave a test data accuracy of 77.4%"""
        for word in reviewWords:
            if word in vocabulary:
                numberOfWords = numberOfWords + 1.
                featureVector = np.add(featureVector, np.square(model[word]))

        featureVector = np.divide(featureVector, numberOfWords)
        featureVector = np.sqrt(featureVector)

    if type == 3:
        """This is feature creation using standard deviation of all word vectors which gave a test data accuracy of 77%"""
        featureVec = np.zeros((1, numberOfFeatures), dtype=np.float32)
        for word in reviewWords:
            if word in vocabulary:
                featureVec = np.add(featureVec, np.square(np.subtract(model[word], featureVector)))

        featureVector = np.sqrt(np.divide(featureVec, numberOfWords))

    return featureVector

In [16]:
def setUpDataForWordVectors(tokenizer):
    """
        This function reads data from labeledTrainData.tsv, unlabeledTrainData.tsv and testData.tsv and accumalates reviews
        from all the three to be used for training the word2vec model.
    """
    trainingData = []

    trainingData += prepareTrainingData(train['review'], tokenizer)

    trainingData += prepareTrainingData(test['review'], tokenizer)

    return trainingData

In [17]:
def setupFeatures(trainingData, testData, model, numberOfFeatures=300, type=1):
    """This function sets up the vectors for the training as well as the test data"""
    stopWords = set(stopwords.words("english"))

    cleanReviews = []

    for review in trainingData["review"]:
        review = re.sub("http:[a-zA-Z0-9/\.]*", " ", review)
        cleanReviews.append(cleanReviewSentences(review, True, stopWords))

    trainingDataVectors = createFeatureForReviews(cleanReviews, model, numberOfFeatures, type)

    cleanTestData = []

    for review in testData["review"]:
        review = re.sub("http:[a-zA-Z0-9/\.]*", " ", review)
        cleanTestData.append(cleanReviewSentences(review, True, stopWords))

    testDataVectors = createFeatureForReviews(cleanTestData, model, numberOfFeatures, type)

    return trainingDataVectors, testDataVectors


In [18]:
def clusterWords(model, numberOfWordsPerCluster=5):
    """This function clusters all the words of the word2vec model vocabulary and returns a dictionary of cluster assignments"""

    wordVectors = model.wv.syn0
    numberOfClusters = int(model.wv.syn0.shape[0] / numberOfWordsPerCluster)

    kmeansClusteringObject = KMeans(n_clusters=numberOfClusters)

    clusterAssignments = kmeansClusteringObject.fit_predict(wordVectors)

    clusterAssignments = dict(zip(model.wv.index2word, clusterAssignments))

    return clusterAssignments

In [19]:
def createFeaturesWithClusters(reviewWords, clusterAssignments, numberOfClusters):
    """
        This function creates the feature vector of the passed review by computing the number of words in the review
        belonging to each cluster.
    """
    featureVector = np.zeros((1, numberOfClusters), dtype=np.float32)[0]

    for word in reviewWords:
        if word in clusterAssignments:
            index = clusterAssignments[word]
            featureVector[index] += 1

    return featureVector

In [20]:
def createFeatureForReviewsUsingClusters(dataFrame, clusterAssignments):
    """
        This function first cleans all the reviews and calls createFeaturesWithClusters for each review to create the feature matrix
        This approach resulted in the best accuracy of 85.1% on the test set.
    """
    stopWords = set(stopwords.words("english"))
    cleanReviews = []

    for review in dataFrame['review']:
        review = re.sub("http:[a-zA-Z0-9/\.]*", " ", review)
        cleanReviews.append(cleanReviewSentences(review, True, stopWords))

    reviewNumber = 0
    numberOfReviews = len(cleanReviews)

    numberOfClusters = max(clusterAssignments.values()) + 1

    featureVectors = np.zeros((numberOfReviews, numberOfClusters), dtype=np.float32)

    for reviewWords in cleanReviews:
        if reviewNumber % 100 == 0:
            printProgress(((reviewNumber + 1) * 100) / numberOfReviews, "Creating Feature Vectors for reviews ..")

        featureVectors[reviewNumber] = createFeaturesWithClusters(reviewWords, clusterAssignments, numberOfClusters)
        reviewNumber += 1
    printProgress(100, "Creating Feature Vectors for reviews ..")

    return featureVectors

In [21]:
from sklearn.cross_validation import train_test_split
# X: documents
# Y: targets
# Now split the data to training set 80% and test set 20%
training_data, validataion_data = train_test_split(train, test_size = 0.2, random_state = 42)

In [22]:
model = wordVectorTraining(300)
clusterAssignments = clusterWords(model, 5)
print (clusterAssignments)
actualTrainDataVectors = createFeatureForReviewsUsingClusters(training_data, clusterAssignments)
actualTestDataVectors = createFeatureForReviewsUsingClusters(validataion_data, clusterAssignments)


{'i': 1302, 'and': 118, 'the': 1584, 'to': 1362, 'it': 738, 'my': 799, 'a': 409, 'for': 156, 'was': 448, 'have': 491, 'of': 1343, 'on': 1553, 'this': 1584, 'in': 650, 'but': 118, 'me': 725, 'had': 862, 'is': 1084, 'that': 738, 'with': 1131, 't': 697, 'not': 1174, 'so': 1325, 'been': 1215, 'at': 206, 'after': 749, 'no': 1680, 'day': 455, 'am': 304, 'as': 1255, 'about': 279, 'm': 304, 'now': 1059, 'side': 966, 'taking': 555, 'all': 1336, 'has': 187, 've': 491, 's': 1084, 'years': 1307, 'pain': 540, 'effects': 451, 'first': 573, 'take': 1019, 'you': 470, 'only': 743, 'months': 176, 'started': 345, '2': 1640, 'like': 631, 'get': 1138, 'up': 1132, 'be': 175, 'days': 519, 'just': 1453, 'very': 1225, 'time': 1521, 'out': 1655, 'or': 1100, 'from': 957, '3': 1640, 'when': 587, 'can': 1394, 'feel': 626, 'if': 349, 'would': 1355, 'because': 509, 'back': 1566, 'pill': 473, 'more': 597, 'one': 409, 'weeks': 176, 'are': 448, 'week': 455, 'took': 555, 'doctor': 589, 'also': 1103, 'period': 460, 'medi

	Creating Feature Vectors for reviews ..	[          ]  1% Completed.

	Creating Feature Vectors for reviews ..	[          ]  2% Completed.

	Creating Feature Vectors for reviews ..	[          ]  3% Completed.

	Creating Feature Vectors for reviews ..	[          ]  4% Completed.

...

	Creating Feature Vectors for reviews ..	[********  ]  82% Completed.

	Creating Feature Vectors for reviews ..	[********* ]  87% Completed.

	Creating Feature Vectors for reviews ..	[********* ]  92% Completed.

	Creating Feature Vectors for reviews ..	[**********]  97% Completed.

	Creating Feature Vectors for reviews ..	[**********]  100% Completed.

In [38]:
print (actualTrainDataVectors)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [40]:
print (actualTestDataVectors)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [41]:
pos_reviews = train.loc[train['rating'] > 5]
neg_reviews = train.loc[train['rating'] < 5]
print('Postive Reviews:  {:,}'.format(len(pos_reviews)))
print('Negative Reviews:  {:,}'.format(len(neg_reviews)))

Postive Reviews:  113,209
Negative Reviews:  40,075


In [43]:
# additional stopwords
from sklearn.feature_extraction import text 
extra_words = ['ve', 'like', 'got', 'just', 
               'don', 'really', 'said', 'told', 'ok',
               'came', 'went', 'did', 'didn', 'good']
stop_words = text.ENGLISH_STOP_WORDS.union(extra_words)

In [45]:

%%time
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a vectorizer object to generate term document counts
tfidf_pos = TfidfVectorizer(stop_words=stop_words, min_df=10, max_df=0.5, 
                        ngram_range=(1,1), token_pattern='[a-z][a-z]+')

tfidf_neg = TfidfVectorizer(stop_words=stop_words, min_df=10, max_df=0.5, 
                        ngram_range=(1,1), token_pattern='[a-z][a-z]+')

# Get random subset of reviews
np.random.seed(38)
num_reviews = 153284
random_neg = np.random.choice(neg_reviews['review'], size=num_reviews)
random_pos = np.random.choice(pos_reviews['review'], size=num_reviews)

# Get the vectors
neg_vectors = tfidf_neg.fit_transform(random_neg)
pos_vectors = tfidf_pos.fit_transform(random_pos)

Wall time: 17.5 s


In [47]:
# Store TFIDF vectors in a Pandas DataFrame to investigate further
neg_df = pd.DataFrame(neg_vectors.todense(), columns=[tfidf_neg.get_feature_names()])
pos_df = pd.DataFrame(pos_vectors.todense(), columns=[tfidf_pos.get_feature_names()])

# get mean for each column (word): highest means are most important words
col_means_neg = {}
for col in neg_df:
    col_means_neg[col] = neg_df[col].mean()

col_means_pos = {}
for col in pos_df:
    col_means_pos[col] = pos_df[col].mean()

In [50]:
no_top_words = 100

print('Top %d words in POSITIVE reviews:' %no_top_words, end='')
print(sorted(col_means_pos, key=col_means_pos.get, reverse=True)[:no_top_words])
print('\n')

print('Top %d words in NEGATIVE reviews:' %no_top_words, end='')
print(sorted(col_means_neg, key=col_means_neg.get, reverse=True)[:no_top_words])
print('\n')

Top 100 words in POSITIVE reviews:[('day',), ('pain',), ('mg',), ('years',), ('taking',), ('effects',), ('months',), ('days',), ('started',), ('feel',), ('time',), ('works',), ('great',), ('pill',), ('life',), ('medication',), ('medicine',), ('period',), ('weight',), ('weeks',), ('anxiety',), ('week',), ('better',), ('month',), ('took',), ('doctor',), ('bad',), ('quot',), ('worked',), ('sleep',), ('work',), ('control',), ('year',), ('night',), ('tried',), ('little',), ('felt',), ('acne',), ('far',), ('used',), ('ago',), ('depression',), ('drug',), ('having',), ('hours',), ('helped',), ('dose',), ('birth',), ('prescribed',), ('love',), ('lbs',), ('lot',), ('use',), ('best',), ('going',), ('help',), ('lost',), ('skin',), ('using',), ('periods',), ('feeling',), ('recommend',), ('effect',), ('getting',), ('away',), ('severe',), ('times',), ('normal',), ('nausea',), ('gain',), ('amp',), ('long',), ('helps',), ('think',), ('mood',), ('does',), ('cramps',), ('sex',), ('symptoms',), ('try',), 